In [ ]:
# !pip install pandas
# !pip install scikit-learn
# !pip install numpy

In [4]:
from google.colab import drive
drive.mount('/content/drive')

ModuleNotFoundError: No module named 'google.colab'

In [5]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [6]:
X_full = pd.read_csv(r'C:\Users\11 PrO\Downloads\train.csv\train.csv', index_col = 'patient_id')
X_test = pd.read_csv(r'C:\Users\11 PrO\Downloads\test.csv\test.csv', index_col = 'patient_id')

# Removing these columns as more empty values than actual values
X_full.drop(['metastatic_first_novel_treatment', 'metastatic_first_novel_treatment_type'], axis = 1)
X_test.drop(['metastatic_first_novel_treatment', 'metastatic_first_novel_treatment_type'], axis = 1)

# Specifying Target Column
X_full.dropna(axis = 0, subset  = ['treatment_pd'], inplace = True)
y = X_full.treatment_pd
X_full.drop(['treatment_pd'], axis = 1, inplace = True)

#Splitting
X_train, X_valid, y_train, y_valid = train_test_split(X_full, y, train_size = 0.8, test_size = 0.2, random_state = 0)

In [ ]:
print(y_train)
# X_train.head()

In [7]:
#Columns with categorical data
object_cols = [i for i in X_train.columns if
              X_train[i].dtype == "object"]

print(object_cols)
print(len(object_cols)) #13 categorical columns

['patient_race', 'payer_type', 'patient_state', 'patient_gender', 'breast_cancer_diagnosis_code', 'breast_cancer_diagnosis_desc', 'metastatic_cancer_diagnosis_code', 'metastatic_first_treatment', 'metastatic_first_treatment_type', 'metastatic_first_novel_treatment', 'metastatic_first_novel_treatment_type', 'region', 'division']
13


In [8]:
#The categorical columns good for encoding

good_label_cols = [i for i in object_cols if
                  set(X_valid[i]).issubset(set(X_train[i]))]

print(good_label_cols)

['patient_race', 'payer_type', 'patient_state', 'patient_gender', 'metastatic_first_treatment', 'metastatic_first_treatment_type', 'metastatic_first_novel_treatment', 'metastatic_first_novel_treatment_type', 'region', 'division']


In [9]:
#Bad columns... not ideal to one-hot encode
bad_label_cols = list(set(object_cols) - set(good_label_cols))

print(bad_label_cols)

['breast_cancer_diagnosis_code', 'breast_cancer_diagnosis_desc', 'metastatic_cancer_diagnosis_code']


In [10]:
#To get the number of unique values in each columns

object_nunique = list(map(lambda col: X_train[col].nunique(), object_cols))
d = dict(zip(object_cols, object_nunique))

# Print number of unique entries by column, in ascending order
sorted(d.items(), key=lambda x: x[1])

#ideally should encode for values with <10 unique values

[('patient_gender', 1),
 ('metastatic_first_novel_treatment_type', 1),
 ('metastatic_first_novel_treatment', 2),
 ('payer_type', 3),
 ('metastatic_first_treatment_type', 4),
 ('region', 4),
 ('patient_race', 5),
 ('division', 9),
 ('metastatic_cancer_diagnosis_code', 40),
 ('metastatic_first_treatment', 40),
 ('patient_state', 52),
 ('breast_cancer_diagnosis_code', 56),
 ('breast_cancer_diagnosis_desc', 56)]

In [11]:
#Better to One-Hot Encode low_cardinality_cols

low_cardinality_cols = [i for i in object_cols if
                       X_train[i].nunique() < 10]

high_cardinality_cols = list(set(object_cols) - set(low_cardinality_cols))

print(low_cardinality_cols)

['patient_race', 'payer_type', 'patient_gender', 'metastatic_first_treatment_type', 'metastatic_first_novel_treatment', 'metastatic_first_novel_treatment_type', 'region', 'division']


In [ ]:
print(y_train)

In [12]:
from sklearn.preprocessing import OneHotEncoder
OH_Encoder = OneHotEncoder(handle_unknown = 'ignore', sparse_output = False)

#One-Hot Encoding
OH_train_cols = pd.DataFrame(OH_Encoder.fit_transform(X_train[low_cardinality_cols]))
OH_valid_cols = pd.DataFrame(OH_Encoder.transform(X_valid[low_cardinality_cols]))

#One-Hot Encoding removes index names... putting back
OH_train_cols.index = X_train.index
OH_valid_cols.index = X_valid.index

#Numerical column data
num_X_train = X_train.drop(object_cols, axis = 1)
num_X_valid = X_valid.drop(object_cols, axis = 1)

#Final Good Data
OH_X_train_final = pd.concat([num_X_train, OH_train_cols], axis = 1 )
OH_X_valid_final = pd.concat([num_X_valid, OH_valid_cols], axis = 1)

#Ensure columns are string names
OH_X_train_final.columns = OH_X_train_final.columns.astype(str)
OH_X_valid_final.columns = OH_X_valid_final.columns.astype(str)

print(OH_X_train_final)
# print(OH_X_valid_final)

            patient_zip3  patient_age    bmi  breast_cancer_diagnosis_year  \
patient_id                                                                   
381467               265           30    NaN                          2017   
280227                 9           49  27.00                          2017   
535537                85           55  32.77                          2018   
494303               921           82  29.00                          2015   
143491               805           61  24.00                          2016   
...                  ...          ...    ...                           ...   
829183               403           43  35.75                          2018   
552056               507           60    NaN                          2017   
808723               917           58    NaN                          2016   
182516               840           54    NaN                          2016   
515010               960           53  24.00                    

In [13]:
# Imputing null values

from sklearn.impute import SimpleImputer

imputer = SimpleImputer(strategy = 'median')

imputed_X_train = pd.DataFrame(imputer.fit_transform(OH_X_train_final))
imputed_X_valid = pd.DataFrame(imputer.transform(OH_X_valid_final))

#Putting col names back
imputed_X_train.columns = OH_X_train_final.columns
imputed_X_valid.columns = OH_X_valid_final.columns

In [ ]:
print(imputed_X_train)

In [ ]:
print(y_train)

In [14]:
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error
import numpy as np

# function for finding prediction
def score_dataset(X_train, X_valid, y_train, y_valid):
    model= SVR(kernel='linear')
    model.fit(X_train, y_train)
    preds = model.predict(X_valid)
    mse = mean_squared_error(y_valid, preds)
    final_rmse = np.sqrt(mse)
    return final_rmse

In [16]:
print(score_dataset(imputed_X_train, imputed_X_valid, y_train, y_valid))

In [15]:
#TEST DATA...
# One Hot Encode test data and Impute null values

OH_test_cols = pd.DataFrame(OH_Encoder.transform(X_test[low_cardinality_cols]))

#Put names back
OH_test_cols.index = X_test.index

#numerical cols
num_X_test = X_test.drop(object_cols, axis = 1)

#Final good testing data
OH_X_test_final = pd.concat([num_X_test, OH_test_cols], axis = 1 )

#Str col names
OH_X_test_final.columns = OH_X_test_final.columns.astype(str)

# print(OH_X_test_final)

#Imputing values
imputed_X_test = pd.DataFrame(imputer.fit_transform(OH_X_test_final))

#Putting col names back
imputed_X_test.columns = OH_X_test_final.columns

#Final final test data = imputed_X_test

print(imputed_X_test)

       patient_zip3  patient_age    bmi  breast_cancer_diagnosis_year  \
0             600.0         49.0  28.34                        2016.0   
1             455.0         53.0  28.34                        2017.0   
2             234.0         60.0  28.34                        2017.0   
3             925.0         51.0  28.34                        2018.0   
4             910.0         47.0  27.82                        2015.0   
...             ...          ...    ...                           ...   
11840         443.0         70.0  33.00                        2016.0   
11841         212.0         55.0  35.87                        2018.0   
11842         296.0         44.0  28.34                        2018.0   
11843         947.0         70.0  28.34                        2018.0   
11844          67.0         64.0  28.34                        2018.0   

       population      density  age_median  age_under_10  age_10_to_19  \
0         25744.0   981.763077   41.762500     11

In [15]:
model = SVR()
model.fit(imputed_X_train, y_train)
preds_test = model.predict(imputed_X_test)

output = pd.DataFrame({'patient_id': X_test.index,
                       'treatment_pd': preds_test})


c:\Users\11 PrO\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.846e+08, tolerance: 7.713e+04
  model = cd_fast.enet_coordinate_descent(


In [16]:
#Submitting to the competition!
output.to_csv('/content/drive/MyDrive/Colab Notebooks/submission-ADU_E_56.csv.csv', index = False)

OSError: Cannot save file into a non-existent directory: '\content\drive\MyDrive\Colab Notebooks'